## FAISS

In [1]:
import requests
from io import StringIO
import pandas as pd

C:\Users\gowth\anaconda3\envs\ML\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\gowth\anaconda3\envs\ML\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\gowth\anaconda3\envs\ML\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [20]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [3]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [4]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

4802

In [5]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [22]:
# each of these dataset have the same structure, so we loop through each creating our sentences data
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

C:\Users\gowth\AppData\Local\Temp\ipykernel_37868\1208996349.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 191: expected 3 fields, saw 4
Skipping line 206: expected 3 fields, saw 4
Skipping line 295: expected 3 fields, saw 4
Skipping line 695: expected 3 fields, saw 4
Skipping line 699: expected 3 fields, saw 4

C:\Users\gowth\AppData\Local\Temp\ipykernel_37868\1208996349.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 104: expected 3 fields, saw 4
Skipping line 181: expected 3 fields, saw 4
Skipping line 317: expected 3 fields, saw 4
Skipping line 412: expected 3 fields, saw 5
Skipping line 508

In [24]:
data

,0,1,2
0,NaN,Small dog chews on a big stick.,a dog shews on a big stick.
1,NaN,A tennis player hitting the ball.,Two boys splashing in the surf.
2,NaN,a lone snowboarder in the middle of a snowy gust,A snowboarder is throwing up snow as he rides ...
3,4.4,A pair of dogs playing with a purple ball.,Two dogs play with purple football.
4,0.6,a bird lands in the water.,a boat floats in the water.
...,...,...,...
1495,NaN,A man doing a trick on a skateboard.,A man in mid air on a skateboard.
1496,NaN,A young girl in swim goggles does the backstro...,A girl does the backstroke in the pool.
1497,5.0,A deer jumps a fence.,A deer is jumping over a fence.
1498,1.0,A young girl dressed in a Minnie mouse outfit ...,a man wearing a white suit holding a newspaper...


In [7]:
len(set(sentences))

14505

In [8]:
# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [9]:
!pip install -U sentence_transformers

In [10]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
# create sentence embeddings
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(14504, 768)

## IndexFlatL2

IndexFlatL2 measures the L2 (or Euclidean) distance between all given points between our query vector, and the vectors loaded into the index. It’s simple, very accurate, but not too fast.

`conda install -c conda-forge faiss` run this command in anaconda prompt in particulare env

In [11]:
import faiss

In [12]:
d = sentence_embeddings.shape[1]
d

768

In [13]:
index = faiss.IndexFlatL2(d)

In [14]:

index.is_trained

True

In [15]:
index.add(sentence_embeddings)

In [16]:
index.ntotal

14504

In [17]:
#Then search given a query `xq` and number of nearest neigbors to return `k`
k = 4
xq = model.encode(["Someone sprints with a football"])

In [18]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 4216 12043  1604 12114]]
CPU times: total: 0 ns
Wall time: 11 ms


In [25]:
data['sentence_A'].iloc[[4586, 10252, 12465, 190]]

KeyError: 'sentence_A'

In [27]:
import numpy as np
# we have 4 vectors to return (k) - so we initialize a zero array to hold them
vecs = np.zeros((k, d))
# then iterate through each ID from I and add the reconstructed vector to our zero-array
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [28]:
vecs.shape

(4, 768)

In [29]:
vecs[0][:100]

array([ 0.01627028,  0.22325905, -0.15037398, -0.30747241, -0.27122438,
       -0.10593199, -0.06460934,  0.04738167, -0.73349017, -0.3765769 ,
       -0.76762819,  0.16902882,  0.53107643,  0.51176673,  1.14415872,
       -0.08562922, -0.67240071, -0.96637076,  0.02545458, -0.21559815,
       -1.25656605, -0.82982183, -0.09824991, -0.21850841,  0.50610214,
        0.10527936,  0.50396854,  0.65242964, -1.39458716,  0.65847456,
       -0.21525349, -0.22487466,  0.81818324,  0.08464271, -0.76141697,
       -0.28928289, -0.09825834, -0.73046178,  0.07855746, -0.84354657,
       -0.59242111,  0.7747137 , -1.20920551, -0.22757959, -1.30733573,
       -0.23081468, -1.31322539,  0.01629057, -0.97285479,  0.19308209,
        0.47424558,  1.18920851, -1.96741331, -0.70061141, -0.29638693,
        0.6053369 ,  0.6240744 , -0.70340413, -0.86754227,  0.17673187,
       -0.19170512, -0.02952   ,  0.22623567, -0.16695452, -0.80402517,
       -0.45918921,  0.69675559, -0.24928206, -1.01478672, -0.92

### IndexIVFFlat

In [30]:
nlist = 50  # how many cells
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [31]:
index.is_trained

False

In [32]:
index.train(sentence_embeddings)
index.is_trained  # check if index is now trained

True

In [33]:
index.add(sentence_embeddings)
index.ntotal  # number of embeddings indexed

14504

In [34]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 4216 12043  4917   422]]
CPU times: total: 0 ns
Wall time: 8.58 ms


In [36]:
#We can increase the number of nearby cells to search too with `nprobe`.



UsageError: Line magic function `%%time` not found.


In [38]:
%%time
index.nprobe = 10

D, I = index.search(xq, k)  # search
print(I)

[[ 4216 12043  1604 12114]]
CPU times: total: 0 ns
Wall time: 2.94 ms


In [39]:
index.make_direct_map()

In [40]:
index.reconstruct(7460)[:100]

array([ 0.33689758, -0.7034185 ,  2.234036  ,  0.5615955 ,  0.42795908,
        0.40639818, -0.31032157,  0.85283655,  0.2588712 , -0.08674854,
       -0.37873736,  0.44926167,  0.8726148 ,  0.35175976, -0.23612525,
        0.43953326, -0.43997982, -0.6856005 ,  0.27687788, -0.736738  ,
        0.01824866,  0.91251284, -0.21928363, -0.56185853,  0.16488653,
       -1.1916667 , -0.23858562, -1.3323933 , -0.46916163, -0.33261868,
        0.20164698, -0.35874274,  0.0182787 , -0.44235042, -0.46859777,
        0.788731  ,  0.3963751 ,  0.43274668,  0.27854833, -0.08967862,
       -0.39413035, -0.05563438,  1.1465826 ,  0.23450384, -0.34182662,
       -0.91904444,  0.5342743 ,  0.16366613, -0.1208844 , -0.14704876,
       -0.97097623, -0.22479868,  0.951626  ,  0.47216034, -0.47807124,
        0.44931424,  0.33649278, -1.3891865 , -0.14561898,  0.65750325,
        0.5134518 , -0.29922578,  0.9637563 ,  0.599508  ,  0.07742058,
        0.10308068,  0.01886696, -0.90011907, -0.05712999, -0.17

## Quantization

Faiss comes with the ability to compress our vectors using Product Quantization (PQ).

IVF allowed us to approximate by reducing the scope of our search, PQ approximates the distance/similarity calculation instead.

In [ ]:
--->Three steps of product quantization
* We split the original vector into several subvectors.
* For each set of subvectors, we perform a clustering operation — creating multiple centroids for each sub-vector set.
* In our vector of sub-vectors, we replace each sub-vector with the ID of it’s nearest set-specific centroid.


### IndexIVFPQ

In [41]:
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits) 

In [42]:
index.is_trained

False

In [43]:
index.train(sentence_embeddings)

In [44]:
index.add(sentence_embeddings)

In [45]:
[f'{i}: {sentences[i]}' for i in I[0]]

['4216: A group of football players is running in the field',
 '12043: A group of people playing football is running in the field',
 '1604: Two groups of people are playing football',
 '12114: A person playing football is running past an official carrying a football']